# DanishGridInsight Project: Step 01 Feature Engineering Pipeline

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 📖 TABLE OF CONTENTS

- [Section 1]()
  - [Subsection 1]()
    - [Subsubsection 1]()
    - [Subsubsection 2]()
  - [Subsection 2]()
    - [Subsubsection 1]()
    - [Subsubsection 2]()
- [Section 2]()
  - [Subsection 1]()
    - [Subsubsection 1]()
    - [Subsubsection 2]()
  - [Subsection 2]()
    - [Subsubsection 1]()
    - [Subsubsection 2]()

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

# 1. Data Source

We used an open API that provides hourly energy consumption values for all the energy consumer types within Denmark.

They provide an intuitive interface where you can easily query and visualize the data. [You can access the data here](https://www.energidataservice.dk/tso-electricity/ConsumptionDE35Hour).

The data has 4 main attributes:

- **Hour UTC:** the UTC datetime when the data point was observed.

- **Price Area:** Denmark is divided into two price areas: DK1 and DK2 — divided by the Great Belt. DK1 is west of the Great Belt, and DK2 is east of the Great Belt.

- **Consumer Type:** The consumer type is the Industry Code DE35, owned and maintained by Danish Energy, a non-commercial lobby organization for Danish energy companies. The code is used by Danish energy companies.

- **Total Consumption:** Total electricity consumption in kWh

The data points have an hourly resolution. For example: "2023—04—15 21:00Z", "2023—04—15 20:00Z", "2023—04—15 19:00Z", etc.

**We will model the data as multiple time series. Each unique price area and consumer type tuple represents its unique time series.**

Thus, **we will build a model that independently forecasts the energy consumption for the next 24 hours for every time series.**

**Note 1:** The observations have a lag of 15 days! But for our demo use case, that is not a problem, as we can simulate the same steps as it would be in real-time.

**Note 2:** [Above dataset](https://www.energidataservice.dk/tso-electricity/ConsumptionDE35Hour) is no longer updated and will be removed from Energi Data Service in December 2023. For data after June 2023, [please find data here](https://www.energidataservice.dk/tso-electricity/ConsumptionDK3619codehour). The observations for this new dataset have a lag of 8 days & 11 hours.

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)

Difficulty: ${\color{green}{Easy}}$
Difficulty: ${\color{orange}{Medium}}$
Difficulty: ${\color{red}{Hard}}$

In [ ]:
# Deep Learning as subset of ML

from IPython import display
display.Image("data/images/Step_01_Feature_Engineering_Pipeline-01.png")

![rainbow](https://github.com/ancilcleetus/My-Learning-Journey/assets/25684256/839c3524-2a1d-4779-85a0-83c562e1e5e5)